<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install yfinance==0.2.59

  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.31.1
    Uninstalling protobuf-6.31.1:
      Successfully uninstalled protobuf-6.31.1


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.65
    Uninstalling yfinance-0.2.65:
      Successfully uninstalled yfinance-0.2.65
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-08-11 13:23:08
-------------------
qualified stocks: 93
with latest results: 28
still star stocks: 18
-------------------
Initial Investment:  1.20 C
CY Investment:  1.47 C
Reserve:  2.51 L
Current:  1.32 C
-------------------
Today PnL: -22.18 K (-0.17%)
Current PnL: -22.74 L (-15.46%)
CY Booked + Current PnL: -14.47 L (-9.84%)
-------------------
Total profit:  2.07 L
Total loss:  -24.80 L
-------------------
Total Booked + Current PnL: 12.51 L (10.42%)
Total Booked PnL: 35.25 L (29.36%)
Curr Year Booked PnL: 8.27 L (6.26%)
Prev Year Booked PnL: 26.98 L (22.48%)
Est FTT:  2.14 C
Est FTT PnL: 82.21 L (62.22%)
Deployed:  1.20 C
Current:  1.32 C
CAGR/XIRR %: 6.18%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'AvgCost', 'FTT', 'Dev%_PE', 'Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
55,MEDANTA,1087.93,1486.00,13.28,H-MC,13.92,161716.0,35516.0,10657.0,-2.04,28.14,6.59,36.59,111.0,3.33,1.25,44.18,XY24,NTT,HEALTHCARE
68,SBIN,760.30,863.00,-12.30,M-LC,4.00,217764.0,16284.0,10932.0,2.17,8.08,5.02,13.51,61.0,1.49,1.68,21.70,XY25,NTT,BANKS
36,HINDALCO,651.95,756.01,-21.55,H-LC,5.72,107752.0,3440.0,13210.0,0.10,3.30,12.26,15.96,11.0,0.26,0.83,20.75,X5K,ATH,METALS
83,TTKPRESTIG,769.29,770.00,93.81,M-SC,10.94,83467.0,-17310.0,17403.0,0.53,-17.18,20.85,0.09,243.0,-0.99,0.64,9.66,OX40N,NTT,DURABLES
51,LALPATHLAB,2838.53,3545.00,-26.54,H-SC,8.67,219395.0,26375.0,21654.0,1.13,13.66,9.87,24.89,134.0,1.22,1.69,40.20,X40N,NTT,HEALTHCARE


In [10]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
41,ICICIPRULI,600.83,790.00,-20.31,H-MC,2.11,137103.0,1315.0,41433.0,-0.56,0.97,30.22,31.48,109.0,0.03,1.06,13.20,X40,ATH,INSURANCE
25,DABUR,505.20,735.00,-9.42,H-MC,4.25,197145.0,117.0,89504.0,-0.34,0.06,45.40,45.49,103.0,0.00,1.52,13.11,XY24,BTT,FMCG
58,PGHH,13388.00,18062.58,-31.21,H-MC,6.54,199260.0,-1560.0,71674.0,0.46,-0.78,35.97,34.92,82.0,-0.02,1.54,3.86,X40,ATH,FMCG
92,WIPRO,243.46,420.00,-15.15,M-LC,6.67,149482.0,-1463.0,110916.0,0.86,-0.97,74.20,72.51,54.0,-0.01,1.15,4.99,XR,NTT,IT


In [11]:
# top 5 to enter based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
21,CIPLA,1492.70,1795.0,-22.87,H-LC,1.99,205719.0,1219.0,40197.0,1.06,0.60,19.54,20.25,10.0,0.03,1.59,9.38,X40N,BTT,PHARMA
5,ASIANPAINT,2961.56,4250.0,-13.06,H-LC,2.42,169490.0,-31896.0,119507.0,0.68,-15.84,70.51,43.51,27.0,-0.27,1.31,17.98,X40,BTT,PAINTS
53,LTIM,5564.16,7230.2,-8.09,H-LC,3.58,182088.0,-18222.0,78207.0,0.92,-9.10,42.95,29.94,17.0,-0.23,1.40,25.80,X200,ATH,IT
57,NESTLEIND,2268.60,2755.0,-16.62,H-LC,4.12,128290.0,-137136.0,194039.0,-1.86,-51.67,151.25,21.44,8.0,-0.71,0.99,3.75,XY25,NTT,FMCG
4,APOLLOHOSP,7098.95,8285.0,-21.39,H-LC,5.44,156970.0,793.0,25304.0,0.71,0.51,16.12,16.71,19.0,0.03,1.21,17.88,X40N,BTT,HEALTHCARE


In [12]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
50,KPIGREEN,497.21,731.35,11.71,H-SC,13.15,133031.0,7734.0,51270.0,4.28,6.17,38.54,47.09,141.0,0.15,1.03,66.50,MH,ATH,POWER
79,TATAMOTORS,761.77,1065.00,-24.80,M-LC,5.51,237366.0,-39157.0,149232.0,3.19,-14.16,62.87,39.81,55.0,-0.26,1.83,13.76,XY24,NTT,AUTO
71,SIEMENS,4406.90,7969.85,-3.25,H-LC,36.11,77775.0,-32397.0,121469.0,2.61,-29.41,156.18,80.85,23.0,-0.27,0.60,14.55,AR,ATH,ELECTRICAL
20,CERA,7989.07,9475.00,-20.05,H-SC,3.06,114174.0,-29629.0,56379.0,2.22,-20.60,49.38,18.60,148.0,-0.53,0.88,25.99,OX40N,NTT,CERAMICS
68,SBIN,760.30,863.00,-12.30,M-LC,4.00,217764.0,16284.0,10932.0,2.17,8.08,5.02,13.51,61.0,1.49,1.68,21.70,XY25,NTT,BANKS


In [13]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
43,INDIGOPNTS,1407.73,1408.00,104.24,M-SC,3.71,133920.0,-40639.0,40672.0,-6.28,-23.28,30.37,0.02,221.0,-1.00,1.03,16.13,OX40N,NTT,PAINTS
90,VOLTAS,1278.28,1918.49,-12.41,H-MC,8.12,183900.0,-7842.0,103867.0,-6.05,-4.09,56.48,50.08,101.0,-0.08,1.42,2.43,XY25,ATH,AC
81,TITAGARH,1097.23,1548.00,-14.37,H-SC,6.21,140516.0,-51499.0,130385.0,-5.97,-26.82,92.79,41.08,156.0,-0.39,1.08,18.93,XY24,NTT,ENGINEERING
52,LAOPALA,369.40,464.00,104.25,H-SC,3.17,64838.0,-36008.0,61836.0,-4.00,-35.71,95.37,25.61,144.0,-0.58,0.50,20.21,AR,NTT,CERAMICS
29,EASEMYTRIP,18.12,26.40,-10.77,M-SC,22.95,85870.0,-90945.0,171740.0,-3.40,-51.44,200.00,45.70,197.0,-0.53,0.66,0.00,XY24,NTT,TRAVEL


In [14]:
# OX40N stocks
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
83,TTKPRESTIG,769.29,770.0,93.81,M-SC,10.94,83467.0,-17310.0,17403.0,0.53,-17.18,20.85,0.09,243.0,-0.99,0.64,9.66,OX40N,NTT,DURABLES
84,UJJIVANSFB,52.77,60.0,82.03,H-SC,20.59,115182.0,-27297.0,46821.0,-3.27,-19.16,40.65,13.70,163.0,-0.58,0.89,35.95,OX40N,NTT,BANKS
20,CERA,7989.07,9475.0,-20.05,H-SC,3.06,114174.0,-29629.0,56379.0,2.22,-20.60,49.38,18.60,148.0,-0.53,0.88,25.99,OX40N,NTT,CERAMICS
78,TATAELXSI,7332.28,9161.0,-19.68,H-MC,2.34,79996.0,-22656.0,48262.0,-0.79,-22.07,60.33,24.94,99.0,-0.47,0.62,21.60,OX40N,NTT,IT
72,SIS,477.00,528.0,2241.97,H-SC,7.06,63002.0,-18088.0,26757.0,-0.58,-22.31,42.47,10.69,155.0,-0.68,0.49,27.68,OX40N,NTT,MISC


In [15]:
# top 5 to accumulate
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
80,TCS,3794.03,4998.00,-28.36,H-LC,15.52,237409.0,-58525.0,152440.0,0.24,-19.78,64.21,31.73,2.0,-0.38,1.83,1.36,X40,BTT,IT
45,INFY,1461.46,2275.00,-21.75,H-LC,9.82,255594.0,-6007.0,151618.0,0.34,-2.30,59.32,55.67,4.0,-0.04,1.97,3.67,X40,BTT,IT
89,VBL,492.64,671.64,-10.71,H-LC,10.94,271394.0,6354.0,89940.0,1.71,2.40,33.14,36.33,5.0,0.07,2.09,16.02,X40N,ATH,FMCG
1,ABB,5551.76,7934.00,-41.91,H-LC,9.72,197340.0,-19179.0,112089.0,0.76,-8.86,56.80,42.91,7.0,-0.17,1.52,3.35,AR,NTT,ELECTRICAL
57,NESTLEIND,2268.60,2755.00,-16.62,H-LC,4.12,128290.0,-137136.0,194039.0,-1.86,-51.67,151.25,21.44,8.0,-0.71,0.99,3.75,XY25,NTT,FMCG


In [16]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
55,MEDANTA,1087.93,1486.0,13.28,H-MC,13.92,161716.0,35516.0,10657.0,-2.04,28.14,6.59,36.59,111.0,3.33,1.25,44.18,XY24,NTT,HEALTHCARE


In [17]:
# top 5 to monitor
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
55,MEDANTA,1087.93,1486.00,13.28,H-MC,13.92,161716.0,35516.0,10657.0,-2.04,28.14,6.59,36.59,111.0,3.33,1.25,44.18,XY24,NTT,HEALTHCARE
50,KPIGREEN,497.21,731.35,11.71,H-SC,13.15,133031.0,7734.0,51270.0,4.28,6.17,38.54,47.09,141.0,0.15,1.03,66.50,MH,ATH,POWER
15,BLUESTARCO,1646.70,2326.38,2.13,H-MC,7.66,176160.0,11490.0,56477.0,-2.15,6.98,32.06,41.28,91.0,0.20,1.36,15.52,X40N,ATH,AC
19,CAMS,3643.00,5211.76,-7.38,H-SC,2.65,105235.0,3231.0,40694.0,0.36,3.17,38.67,43.06,123.0,0.08,0.81,20.83,X40N,ATH,MISC
25,DABUR,505.20,735.00,-9.42,H-MC,4.25,197145.0,117.0,89504.0,-0.34,0.06,45.40,45.49,103.0,0.00,1.52,13.11,XY24,BTT,FMCG


In [18]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
35,HEROMOTOCO,4311.81,5949.07,-13.93,H-MC,6.54,159705.0,8792.0,48518.0,-0.80,5.83,30.38,37.97,92.0,0.18,1.23,32.60,AR,ATH,AUTO
50,KPIGREEN,497.21,731.35,11.71,H-SC,13.15,133031.0,7734.0,51270.0,4.28,6.17,38.54,47.09,141.0,0.15,1.03,66.50,MH,ATH,POWER
42,INDIAMART,2327.09,4850.92,-51.47,H-SC,12.89,135415.0,12079.0,121684.0,-0.25,9.79,89.86,108.45,119.0,0.10,1.04,34.34,AR,ATH,MISC
91,WHIRLPOOL,1219.98,2270.00,-44.07,M-SC,4.47,94500.0,3002.0,75751.0,-0.36,3.28,80.16,86.07,219.0,0.04,0.73,36.70,XR,NTT,DURABLES
92,WIPRO,243.46,420.00,-15.15,M-LC,6.67,149482.0,-1463.0,110916.0,0.86,-0.97,74.20,72.51,54.0,-0.01,1.15,4.99,XR,NTT,IT


In [19]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
23,COFFEEDAY,59.14,80.00,-53.72,L-SC,22.23,68755.0,-44794.0,84844.0,-0.67,-39.45,123.40,35.27,268.0,-0.53,0.53,68.28,XR,NTT,HOTELS
3,ALKYLAMINE,2347.98,4546.37,-5.18,H-SC,17.34,88219.0,-12744.0,107274.0,-0.61,-12.62,121.60,93.63,149.0,-0.12,0.68,31.40,SR,ATH,CHEMICALS
54,MASFIN,326.60,399.50,-20.05,H-SC,15.89,90690.0,-7290.0,29157.0,-0.03,-7.44,32.15,22.32,152.0,-0.25,0.70,31.53,XR,ATH,FINANCE
84,UJJIVANSFB,52.77,60.00,82.03,H-SC,20.59,115182.0,-27297.0,46821.0,-3.27,-19.16,40.65,13.70,163.0,-0.58,0.89,35.95,OX40N,NTT,BANKS
50,KPIGREEN,497.21,731.35,11.71,H-SC,13.15,133031.0,7734.0,51270.0,4.28,6.17,38.54,47.09,141.0,0.15,1.03,66.50,MH,ATH,POWER


In [20]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
89,VBL,492.64,671.64,-10.71,H-LC,10.94,271394.0,6354.0,89940.0,1.71,2.40,33.14,36.33,5.0,0.07,2.09,16.02,X40N,ATH,FMCG
70,SHALBY,261.39,327.00,931.90,M-SC,7.99,140249.0,-40894.0,86365.0,-0.44,-22.58,61.58,25.10,232.0,-0.47,1.08,12.40,XY24,NTT,HEALTHCARE
51,LALPATHLAB,2838.53,3545.00,-26.54,H-SC,8.67,219395.0,26375.0,21654.0,1.13,13.66,9.87,24.89,134.0,1.22,1.69,40.20,X40N,NTT,HEALTHCARE
35,HEROMOTOCO,4311.81,5949.07,-13.93,H-MC,6.54,159705.0,8792.0,48518.0,-0.80,5.83,30.38,37.97,92.0,0.18,1.23,32.60,AR,ATH,AUTO
55,MEDANTA,1087.93,1486.00,13.28,H-MC,13.92,161716.0,35516.0,10657.0,-2.04,28.14,6.59,36.59,111.0,3.33,1.25,44.18,XY24,NTT,HEALTHCARE


In [21]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,18.48
1,25,41.12
2,50,70.85


In [22]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    39.45
LC    34.08
MC    26.42
Name: CurrAlloc%, dtype: float64

In [23]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     29.01
X40N     15.32
X40      13.44
XY25     10.69
XR        8.98
AR        8.42
OX40N     7.55
X5K       2.23
MH        1.79
X200      1.40
SR        1.12
Name: CurrAlloc%, dtype: float64

In [24]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-LC    25.76
H-SC    24.58
H-MC    23.46
M-SC    13.42
M-LC     7.23
M-MC     2.62
L-SC     1.45
L-LC     1.09
L-MC     0.34
Name: CurrAlloc%, dtype: float64

In [25]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,14.32,-11.03,46.73
IT,12.02,-19.58,79.31
FINANCE,8.30,-21.01,65.78
BANKS,7.65,-14.96,63.77
MISC,5.93,-19.54,88.06
PAINTS,5.66,-19.01,42.46
HEALTHCARE,5.23,3.21,21.23
ELECTRICAL,4.72,-14.68,64.35
AUTO,4.44,-17.52,68.65


In [26]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,2874480.0,23
XR,1130071.0,13
AR,1116024.0,10
X40,828877.0,10
XY25,663282.0,8
X40N,619583.0,12
OX40N,479502.0,10
SR,197831.0,2
X5K,149770.0,2


In [27]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,2827954.0,28
M-SC,1505603.0,17
H-LC,1394689.0,18
H-MC,1337230.0,18
M-LC,440972.0,5
M-MC,337609.0,2
L-SC,273606.0,3
L-MC,57935.0,1
L-LC,45540.0,1


In [28]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                 
H-SC       XY24      945270.0      6
M-SC       XY24      878191.0      7
H-SC       AR        672094.0      5
           XR        532512.0      6
H-LC       X40       520228.0      5
H-MC       XY24      486355.0      5
M-MC       XY24      337609.0      2
H-LC       XY25      254584.0      3
           X40N      250411.0      5
           AR        233558.0      2
H-MC       X40       233014.0      4
           XY25      225557.0      2
H-SC       X40N      224939.0      4
           OX40N     204038.0      4
           SR        197831.0      2
L-SC       XR        183122.0      2
M-LC       XY24      182564.0      2
M-SC       AR        161854.0      2
H-MC       X40N      144233.0      3
M-SC       OX40N     136718.0      4
M-LC       X5K       136560.0      1
M-SC       XY25      126669.0      1
           XR        126536.0      2
H-MC       XR        119050.0      1
M-LC       XR        110916.0      1
L-SC       OX40N      90484.0      1
H-LC       X200       78207.0      1
M-SC       X40        75635.0      1
L-MC       XR         57935.0      1
H-SC       MH         51270.0      1
H-MC       AR         48518.0      1
           OX40N      48262.0      1
L-LC       XY25       45540.0      1
H-LC       XY24       44491.0      1
H-MC       MH         32241.0      1
H-LC       X5K        13210.0      1
M-LC       XY25       10932.0      1

In [29]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 31.0 seconds
